# Описание кейса «Прогнозирование оттока клиентов»
 
Суть бизнес задачи:
Каждый год компания пролонгирует полисы Каско клиентов - физических лиц. Для оптимизации работы со списками на пролонгацию необходимо прогнозировать с какой вероятностью каждый из клиентов пролонгируется и какие факторы на это влияют. В зависимости от этого расставляются приоритеты операторам колл-центра, которые обрабатывают список (осуществляют обзвон клиентов), а также принимаются решения по дополнительной мотивации клиентов к пролонгации.
 
Описание массива:
Дана выборка полисов на пролонгацию в формате CSV-файла с набором полей, характеризующих сам полис, клиента (и его историю страхования) и транспортное средство. Полис на пролонгацию – это полис, период действия заканчивается и который нужно пролонгировать.
Прогнозируемая переменная – факт пролонгации полиса «POLICY_IS_RENEWED», где «1» – клиент пролонгировался, «0» - клиент не пролонгировался.
Массив случайным образом разбит на 2 части: 80% данных – тренировочная выборка, 20% данных – тестовая выборка.
 
Проверка и оценка результата:
Для целей моделирования и проверки данных предоставляется тренировочная и тестовая выборки. В тестовой выборке значения прогнозируемой переменной обнулены.


In [1]:
#импортируем библиотеки и расширяем видимость кол-а столбцов
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV # для определения лучшего параметра
from sklearn.model_selection import cross_val_score
pd.set_option('display.max_columns', 35)

# Первичная обработка данных. Т.к. нету пропусков в численных данных, то их не нужно заполнять. Смотрим на данные и их взаимосвязи

In [2]:
df = pd.read_csv('data.csv', sep = ';') # считываем файл 

In [3]:
df.shape

(96605, 30)

In [4]:
df.head(5)

,DATA_TYPE,POLICY_ID,POLICY_BEGIN_MONTH,POLICY_END_MONTH,POLICY_IS_RENEWED,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,CLIENT_REGISTRATION_REGION,POLICY_PRICE_CHANGE
0,TRAIN,1,1,1,1,39,1,Москва,51,12,Land Rover,Discovery,245.0,0,1283202.0,856,F,0,0,N,N,1,0,0,0.0,0,0,0.0,Тульская,-1.00
1,TRAIN,2,1,1,1,50,5,Москва,35,7,Opel,Zafira,140.0,0,453000.0,N,M,0,0,0,0,1,1,0,0.0,0,4,0.0,Москва,-0.05
2,TRAIN,3,1,1,1,52,6,Москва,41,6,Kia,Soul,129.0,0,647718.0,N,F,0,0,1L,1L,0,0,0,0.0,0,1,12518.0,Московская,-0.07
3,TRAIN,4,1,1,1,50,5,Москва,36,12,Citroen,C-Crosser,170.0,0,757795.0,N,M,0,0,0,0,1,1,0,0.0,0,6,15000.0,Москва,0.05
4,TRAIN,5,1,1,0,52,6,Санкт-Петербург,42,5,Renault,Logan,102.0,0,604500.0,171,F,0,0,N,N,0,0,0,0.0,0,0,50000.0,Ленинградская,0.17


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96605 entries, 0 to 96604
Data columns (total 30 columns):
DATA_TYPE                        96605 non-null object
POLICY_ID                        96605 non-null int64
POLICY_BEGIN_MONTH               96605 non-null int64
POLICY_END_MONTH                 96605 non-null int64
POLICY_IS_RENEWED                96605 non-null int64
POLICY_SALES_CHANNEL             96605 non-null int64
POLICY_SALES_CHANNEL_GROUP       96605 non-null int64
POLICY_BRANCH                    96605 non-null object
POLICY_MIN_AGE                   96605 non-null int64
POLICY_MIN_DRIVING_EXPERIENCE    96605 non-null int64
VEHICLE_MAKE                     96605 non-null object
VEHICLE_MODEL                    96605 non-null object
VEHICLE_ENGINE_POWER             96605 non-null float64
VEHICLE_IN_CREDIT                96605 non-null int64
VEHICLE_SUM_INSURED              96605 non-null float64
POLICY_INTERMEDIARY              96605 non-null object
INSURER_GENDER    

### Категориальные признаки указаны как числовые - 'VEHICLE_IN_CREDIT', 'CLIENT_HAS_DAGO', 'CLIENT_HAS_OSAGO', 'POLICY_HAS_COMPLAINTS', 'POLICY_SALES_CHANNEL', 'POLICY_SALES_CHANNEL_GROUP', 'VEHICLE_IN_CREDIT', 'POLICY_COURT_SIGN', 'POLICY_HAS_COMPLAINTS'. Нужно будет их тип изменить для дальнейшей корректной векторизации данных

In [6]:
#В моем понимании начало и конец действия полиса без указания года не несут в себе информации.
#В связи с чем удалил их из расмотрения
df.drop(['POLICY_BEGIN_MONTH', 'POLICY_END_MONTH'], axis = 1, inplace = True) 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96605 entries, 0 to 96604
Data columns (total 28 columns):
DATA_TYPE                        96605 non-null object
POLICY_ID                        96605 non-null int64
POLICY_IS_RENEWED                96605 non-null int64
POLICY_SALES_CHANNEL             96605 non-null int64
POLICY_SALES_CHANNEL_GROUP       96605 non-null int64
POLICY_BRANCH                    96605 non-null object
POLICY_MIN_AGE                   96605 non-null int64
POLICY_MIN_DRIVING_EXPERIENCE    96605 non-null int64
VEHICLE_MAKE                     96605 non-null object
VEHICLE_MODEL                    96605 non-null object
VEHICLE_ENGINE_POWER             96605 non-null float64
VEHICLE_IN_CREDIT                96605 non-null int64
VEHICLE_SUM_INSURED              96605 non-null float64
POLICY_INTERMEDIARY              96605 non-null object
INSURER_GENDER                   96605 non-null object
POLICY_CLM_N                     96605 non-null object
POLICY_CLM_GLT_N

In [8]:
#Выбрал признаки которые по моему должны быть категориальными а записаны как числовые
#Меняю их типы в датафрейме
categoric_name = ['VEHICLE_IN_CREDIT', 'CLIENT_HAS_DAGO', 'CLIENT_HAS_OSAGO', 'POLICY_HAS_COMPLAINTS',
                 'POLICY_SALES_CHANNEL', 'POLICY_SALES_CHANNEL_GROUP', 'VEHICLE_IN_CREDIT',
                 'POLICY_COURT_SIGN', 'POLICY_HAS_COMPLAINTS']
for col in categoric_name:
    df[col] = df[col].astype('object')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96605 entries, 0 to 96604
Data columns (total 28 columns):
DATA_TYPE                        96605 non-null object
POLICY_ID                        96605 non-null int64
POLICY_IS_RENEWED                96605 non-null int64
POLICY_SALES_CHANNEL             96605 non-null object
POLICY_SALES_CHANNEL_GROUP       96605 non-null object
POLICY_BRANCH                    96605 non-null object
POLICY_MIN_AGE                   96605 non-null int64
POLICY_MIN_DRIVING_EXPERIENCE    96605 non-null int64
VEHICLE_MAKE                     96605 non-null object
VEHICLE_MODEL                    96605 non-null object
VEHICLE_ENGINE_POWER             96605 non-null float64
VEHICLE_IN_CREDIT                96605 non-null object
VEHICLE_SUM_INSURED              96605 non-null float64
POLICY_INTERMEDIARY              96605 non-null object
INSURER_GENDER                   96605 non-null object
POLICY_CLM_N                     96605 non-null object
POLICY_CLM_GL

In [9]:
'''
Числовой признак записан как категориальный. Т.к. возможно это кол-о лет в течении которых человек продлевает
полис, то пропуски N лучше заменить на 0. Т.к. мы ничего не знаем об этом человеке
'''
mask = df['POLICY_YEARS_RENEWED_N'] == 'N'
df.loc[mask, 'POLICY_YEARS_RENEWED_N']  = '0'
df[mask].head()

,DATA_TYPE,POLICY_ID,POLICY_IS_RENEWED,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,CLIENT_REGISTRATION_REGION,POLICY_PRICE_CHANGE
3111,TRAIN,3112,1,15,1,Москва,45,20,Toyota,RAV4,152.0,0,443000.0,1028,F,n/d,n/d,N,N,0,1,0,0.0,0,0,10000.0,Москва,0.05
5830,TRAIN,5831,0,59,3,Санкт-Петербург,55,34,Nissan,X-Trail,141.0,0,792800.0,1219,M,n/d,n/d,N,N,0,0,0,0.0,0,0,15000.0,Санкт-Петербург,-1.00
9655,TEST,9656,0,53,6,Санкт-Петербург,42,15,Peugeot,4007,170.0,0,800000.0,N,F,n/d,n/d,N,N,1,1,0,0.0,0,0,0.0,Санкт-Петербург,0.15
10223,TRAIN,10224,1,52,6,Санкт-Петербург,36,8,BMW,X1,184.0,0,1214000.0,6,M,n/d,n/d,N,N,0,0,0,1.0,0,0,0.0,Санкт-Петербург,0.16
10241,TRAIN,10242,1,60,3,Москва,72,53,Renault,Duster,102.0,1,461161.0,N,F,n/d,n/d,N,N,0,0,0,0.0,0,0,0.0,Москва,0.19


In [10]:
df['POLICY_YEARS_RENEWED_N'] = df['POLICY_YEARS_RENEWED_N'].astype('float')# меняем тип на количественный

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96605 entries, 0 to 96604
Data columns (total 28 columns):
DATA_TYPE                        96605 non-null object
POLICY_ID                        96605 non-null int64
POLICY_IS_RENEWED                96605 non-null int64
POLICY_SALES_CHANNEL             96605 non-null object
POLICY_SALES_CHANNEL_GROUP       96605 non-null object
POLICY_BRANCH                    96605 non-null object
POLICY_MIN_AGE                   96605 non-null int64
POLICY_MIN_DRIVING_EXPERIENCE    96605 non-null int64
VEHICLE_MAKE                     96605 non-null object
VEHICLE_MODEL                    96605 non-null object
VEHICLE_ENGINE_POWER             96605 non-null float64
VEHICLE_IN_CREDIT                96605 non-null object
VEHICLE_SUM_INSURED              96605 non-null float64
POLICY_INTERMEDIARY              96605 non-null object
INSURER_GENDER                   96605 non-null object
POLICY_CLM_N                     96605 non-null object
POLICY_CLM_GL

# Обработка редких категорий

### Т.к. в момент когда мы категориальные данные векторезуем, то в итоге если признак содержит N категорий мы можем увеличить размерность нашего вектора на N. Поэтому нужно объединить редкие категории в один для уменьшения размености итогового вектора 

In [12]:
#Выводим категории с частотами встречаемосит по каждой кат-ому признаку
categorical_columns = [col for col in df.columns if df[col].dtype.name == 'object']
for col in categorical_columns:
    print(col)
    print(df[col].value_counts(), '\n')

DATA_TYPE
TRAIN    77407
TEST     19198
Name: DATA_TYPE, dtype: int64 

POLICY_SALES_CHANNEL
52    24728
53    21070
50    16176
55    11397
54     4897
59     2923
10     2332
6      1093
1       974
13      867
14      747
15      735
8       676
4       579
60      570
11      487
39      451
18      444
2       440
23      413
16      359
46      268
45      262
3       250
41      242
40      215
49      210
63      193
22      183
43      160
      ...  
7       144
27      128
62      113
47      107
17       98
21       97
33       96
31       92
32       88
26       85
25       80
42       74
20       66
44       66
9        60
24       60
5        55
35       53
28       48
29       37
36       30
37       29
57       24
12       20
48       15
61       15
30       14
58        7
38        2
56        1
Name: POLICY_SALES_CHANNEL, Length: 63, dtype: int64 

POLICY_SALES_CHANNEL_GROUP
6    62093
5    16176
1     9860
4     4211
3     3621
8      420
2      200
7       24
Name:

## Решение по объединению категорий выбирал на свое усмотрение. Пропуски в кат-ых признаках обозначены как N. Их можно отнести к отдельной категории, поэтому не заполнял и не менял их
### Признаки которые содержат редкие категории:
#### 1) POLICY_SALES_CHANNEL (объединить наблюдения с частотой менее 1000 в категорию "OTHER")
#### 2) POLICY_SALES_CHANNEL_GROUP (объединить наблюдения с частотой менее 3000 в категорию "OTHER")
#### 3) VEHICLE_MAKE (объединить наблюдения с частотой менее 600 в категорию "OTHER")
#### 4) VEHICLE_MODEL(объединить наблюдения с частотой менее 600 в категорию "OTHER")
#### 5) POLICY_INTERMEDIARY(объединить наблюдения с частотой менее 600 в категорию "OTHER"). 
#### 6) POLICY_CLM_N (объединить наблюдения с частотой менее 600 в категорию "OTHER")
#### 7) POLICY_CLM_GLT_N (объединить наблюдения с частотой менее 600 в категорию "OTHER")
#### 8) POLICY_PRV_CLM_GLT_N (объединить наблюдения с частотой менее 600 в категорию "OTHER").
#### 9) POLICY_PRV_CLM_N (объединить наблюдения с частотой менее 800 в категорию "OTHER"). 
#### 10) CLIENT_REGISTRATION_REGION (объединить наблюдения с частотой менее 200 в категорию "OTHER"). 

In [13]:
#список признаков где нужно будет объединить данные с частотами менее 600
col_600 = ['VEHICLE_MAKE', 'VEHICLE_MODEL', 'POLICY_INTERMEDIARY',
          'POLICY_CLM_N', 'POLICY_CLM_GLT_N', 'POLICY_PRV_CLM_GLT_N']

In [14]:
#объединяю категории по признаку POLICY_SALES_CHANNEL с частотами менее 1000 в одну категорию "77777"
print(df['POLICY_SALES_CHANNEL'].value_counts(dropna = False).head(10))
df.loc[df['POLICY_SALES_CHANNEL'].value_counts()[df['POLICY_SALES_CHANNEL']].values < 1000, 'POLICY_SALES_CHANNEL'] = '77777'
print('After:\n', df['POLICY_SALES_CHANNEL'].value_counts(dropna = False))
print('type POLICY_SALES_CHANNEL', df['POLICY_SALES_CHANNEL'].dtype.name)

52    24728
53    21070
50    16176
55    11397
54     4897
59     2923
10     2332
6      1093
1       974
13      867
Name: POLICY_SALES_CHANNEL, dtype: int64
After:
 52       24728
53       21070
50       16176
77777    11989
55       11397
54        4897
59        2923
10        2332
6         1093
Name: POLICY_SALES_CHANNEL, dtype: int64
type POLICY_SALES_CHANNEL object


In [15]:
#объединяю категории по признаку POLICY_SALES_CHANNEL_GROUP с частотами менее 3000 в одну категорию "77777"
print(df['POLICY_SALES_CHANNEL_GROUP'].value_counts(dropna = False).head(10))
df.loc[df['POLICY_SALES_CHANNEL_GROUP'].value_counts()[df['POLICY_SALES_CHANNEL_GROUP']].values < 3000, 
       'POLICY_SALES_CHANNEL_GROUP'] = '77777'
print('After:\n', df['POLICY_SALES_CHANNEL_GROUP'].value_counts(dropna = False))
print('type POLICY_SALES_CHANNEL_GROUP', df['POLICY_SALES_CHANNEL_GROUP'].dtype.name)

6    62093
5    16176
1     9860
4     4211
3     3621
8      420
2      200
7       24
Name: POLICY_SALES_CHANNEL_GROUP, dtype: int64
After:
 6        62093
5        16176
1         9860
4         4211
3         3621
77777      644
Name: POLICY_SALES_CHANNEL_GROUP, dtype: int64
type POLICY_SALES_CHANNEL_GROUP object


In [16]:
#Прохожу по всем кат-ым признакам внесенных в список col_600
#И по каждому признаку объединяю категории с частотами менее 600 в одну категорию "77777"
for col in col_600:    
    print(df[col].value_counts(dropna = False).head(10))
    df.loc[df[col].value_counts()[df[col]].values < 600,
           col] = '77777'
    print('After:\n', df[col].value_counts(dropna = False))
    print('type', col, df[col].dtype.name)

Kia           11050
Hyundai        9168
Toyota         8866
Renault        8733
Ford           8127
Mitsubishi     7169
Nissan         6514
Volkswagen     5178
BMW            4744
Skoda          4054
Name: VEHICLE_MAKE, dtype: int64
After:
 Kia              11050
Hyundai           9168
Toyota            8866
Renault           8733
Ford              8127
Mitsubishi        7169
Nissan            6514
Volkswagen        5178
77777             4880
BMW               4744
Skoda             4054
Audi              2195
Honda             1833
Opel              1789
Volvo             1744
Land Rover        1674
Suzuki            1611
Chevrolet         1306
Mercedes-Benz     1272
Mazda             1225
Subaru            1134
Ssang Yong         939
Lada               789
ВАЗ                611
Name: VEHICLE_MAKE, dtype: int64
type VEHICLE_MAKE object
RAV4         4186
Sportage     3692
Duster       3612
Focus        3196
ix35         2971
Solaris      2799
Outlander    2634
Qashqai      2547
Ceed 

In [17]:
#объединяю категории по признаку POLICY_PRV_CLM_N с частотами менее 800 в одну категорию "77777"
print(df['POLICY_PRV_CLM_N'].value_counts(dropna = False).head(10))
df.loc[df['POLICY_PRV_CLM_N'].value_counts()[df['POLICY_PRV_CLM_N']].values < 800, 
       'POLICY_PRV_CLM_N'] = '77777'
print('After:\n', df['POLICY_PRV_CLM_N'].value_counts(dropna = False))
print('type POLICY_PRV_CLM_N', df['POLICY_PRV_CLM_N'].dtype.name)

0     44587
N     33305
1S     7261
1L     6790
2      3746
3       788
4+      128
Name: POLICY_PRV_CLM_N, dtype: int64
After:
 0        44587
N        33305
1S        7261
1L        6790
2         3746
77777      916
Name: POLICY_PRV_CLM_N, dtype: int64
type POLICY_PRV_CLM_N object


In [18]:
#объединяю категории по признаку CLIENT_REGISTRATION_REGION с частотами менее 200 в одну категорию "77777"
print(df['CLIENT_REGISTRATION_REGION'].value_counts(dropna = False).head(10))
df.loc[df['CLIENT_REGISTRATION_REGION'].value_counts()[df['CLIENT_REGISTRATION_REGION']].values < 200, 
       'CLIENT_REGISTRATION_REGION'] = '77777'
print('After:\n', df['CLIENT_REGISTRATION_REGION'].value_counts(dropna = False))
print('type CLIENT_REGISTRATION_REGION', df['CLIENT_REGISTRATION_REGION'].dtype.name)

Санкт-Петербург    38854
Москва             34420
Московская         12820
Ленинградская       5877
N                   1130
Калужская            275
Тульская             219
Тверская             205
Владимирская         172
Новгородская         138
Name: CLIENT_REGISTRATION_REGION, dtype: int64
After:
 Санкт-Петербург    38854
Москва             34420
Московская         12820
Ленинградская       5877
77777               2805
N                   1130
Калужская            275
Тульская             219
Тверская             205
Name: CLIENT_REGISTRATION_REGION, dtype: int64
type CLIENT_REGISTRATION_REGION object


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96605 entries, 0 to 96604
Data columns (total 28 columns):
DATA_TYPE                        96605 non-null object
POLICY_ID                        96605 non-null int64
POLICY_IS_RENEWED                96605 non-null int64
POLICY_SALES_CHANNEL             96605 non-null object
POLICY_SALES_CHANNEL_GROUP       96605 non-null object
POLICY_BRANCH                    96605 non-null object
POLICY_MIN_AGE                   96605 non-null int64
POLICY_MIN_DRIVING_EXPERIENCE    96605 non-null int64
VEHICLE_MAKE                     96605 non-null object
VEHICLE_MODEL                    96605 non-null object
VEHICLE_ENGINE_POWER             96605 non-null float64
VEHICLE_IN_CREDIT                96605 non-null object
VEHICLE_SUM_INSURED              96605 non-null float64
POLICY_INTERMEDIARY              96605 non-null object
INSURER_GENDER                   96605 non-null object
POLICY_CLM_N                     96605 non-null object
POLICY_CLM_GL

In [20]:
#Смотрю на частоты по каждому кат-му признаку 
categorical_columns = [col for col in df.columns if df[col].dtype.name == 'object']
for col in categorical_columns:
    print(col)
    print(df[col].value_counts(), '\n')

DATA_TYPE
TRAIN    77407
TEST     19198
Name: DATA_TYPE, dtype: int64 

POLICY_SALES_CHANNEL
52       24728
53       21070
50       16176
77777    11989
55       11397
54        4897
59        2923
10        2332
6         1093
Name: POLICY_SALES_CHANNEL, dtype: int64 

POLICY_SALES_CHANNEL_GROUP
6        62093
5        16176
1         9860
4         4211
3         3621
77777      644
Name: POLICY_SALES_CHANNEL_GROUP, dtype: int64 

POLICY_BRANCH
Москва             50697
Санкт-Петербург    45908
Name: POLICY_BRANCH, dtype: int64 

VEHICLE_MAKE
Kia              11050
Hyundai           9168
Toyota            8866
Renault           8733
Ford              8127
Mitsubishi        7169
Nissan            6514
Volkswagen        5178
77777             4880
BMW               4744
Skoda             4054
Audi              2195
Honda             1833
Opel              1789
Volvo             1744
Land Rover        1674
Suzuki            1611
Chevrolet         1306
Mercedes-Benz     1272
Mazda        

In [21]:
df.describe()

,POLICY_ID,POLICY_IS_RENEWED,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_SUM_INSURED,CLAIM_AVG_ACC_ST_PRD,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,POLICY_PRICE_CHANGE
count,96605.000000,96605.000000,96605.000000,96605.000000,96605.000000,9.660500e+04,96605.000000,96605.000000,96605.000000,96605.000000
mean,48303.000000,0.500761,42.580767,34.910357,154.195851,9.806700e+05,3.877110,1.494798,5974.134461,-0.033025
std,27887.605715,0.500002,10.694670,199.928966,54.117543,6.900919e+05,17.635091,1.602090,10418.697183,0.857995
min,1.000000,0.000000,18.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-1.000000
25%,24152.000000,0.000000,34.000000,8.000000,123.000000,5.584960e+05,0.000000,0.000000,0.000000,-0.170000
50%,48303.000000,1.000000,41.000000,14.000000,146.000000,8.090000e+05,0.000000,1.000000,0.000000,0.000000
75%,72454.000000,1.000000,50.000000,19.000000,171.000000,1.163900e+06,0.000000,2.000000,10000.000000,0.050000
max,96605.000000,1.000000,86.000000,2015.000000,2000.000000,9.449000e+06,737.000000,10.000000,120873.000000,60.020000


# Т.к. пропусков(среди численных параметров) в данных нету(категориальные данные уже записан параметр 'N') и не нужно заполнять пропуски можно данные сразу разделить

In [22]:
#делю данные на обучающие и тествые
X_train = df[df['DATA_TYPE'] == 'TRAIN']
X_test = df[df['DATA_TYPE'] == 'TEST '] 
#удаляю не нужные признаки
X_train.drop(['DATA_TYPE', 'POLICY_ID', 'POLICY_IS_RENEWED'], axis = 1, inplace = True)
X_test.drop(['DATA_TYPE', 'POLICY_ID', 'POLICY_IS_RENEWED'], axis = 1, inplace = True)
#ответы на обучающие данные
y_train = df.loc[df['DATA_TYPE'] == 'TRAIN', 'POLICY_IS_RENEWED']
#y_test = df.loc[df['DATA_TYPE'] == 'TEST ', 'POLICY_IS_RENEWED'] тестовые данные обнулены

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [23]:
#проверяю размерности на всякий случай
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

print(X_train.shape[0] + X_test.shape[0], ' - ', df.shape[0])
print(df.shape)

(77407, 25)
(19198, 25)
(77407,)
96605  -  96605
(96605, 28)


In [24]:
# выделяю числовые и категориальные признаки
categorical_columns = [c for c in X_train.columns if X_train[c].dtype.name == 'object']
numerical_columns   = [c for c in X_train.columns if X_train[c].dtype.name != 'object']
categorical_columns

['POLICY_SALES_CHANNEL',
 'POLICY_SALES_CHANNEL_GROUP',
 'POLICY_BRANCH',
 'VEHICLE_MAKE',
 'VEHICLE_MODEL',
 'VEHICLE_IN_CREDIT',
 'POLICY_INTERMEDIARY',
 'INSURER_GENDER',
 'POLICY_CLM_N',
 'POLICY_CLM_GLT_N',
 'POLICY_PRV_CLM_N',
 'POLICY_PRV_CLM_GLT_N',
 'CLIENT_HAS_DAGO',
 'CLIENT_HAS_OSAGO',
 'POLICY_COURT_SIGN',
 'POLICY_HAS_COMPLAINTS',
 'CLIENT_REGISTRATION_REGION']

In [25]:
numerical_columns

['POLICY_MIN_AGE',
 'POLICY_MIN_DRIVING_EXPERIENCE',
 'VEHICLE_ENGINE_POWER',
 'VEHICLE_SUM_INSURED',
 'CLAIM_AVG_ACC_ST_PRD',
 'POLICY_YEARS_RENEWED_N',
 'POLICY_DEDUCT_VALUE',
 'POLICY_PRICE_CHANGE']

In [26]:
X_train[categorical_columns].describe()# смотрю статистику по кат-ым признакам

,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_IN_CREDIT,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,POLICY_HAS_COMPLAINTS,CLIENT_REGISTRATION_REGION
count,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407
unique,9,6,2,24,42,2,27,2,6,6,6,6,2,2,2,2,9
top,52,6,Москва,Kia,77777,0,77777,M,0,0,0,0,0,1,0,0,Санкт-Петербург
freq,19854,49797,40675,8803,28428,52605,30247,48923,54481,57885,35747,38588,56045,42577,77331,76842,31124


In [27]:
X_train.shape

(77407, 25)

In [28]:
#from pandas.plotting import scatter_matrix
#scatter_matrix(df, alpha = 0.5, figsize = (10,10))

In [29]:
X_train.head()

,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,CLIENT_REGISTRATION_REGION,POLICY_PRICE_CHANGE
0,77777,1,Москва,51,12,Land Rover,77777,245.0,0,1283202.0,77777,F,0,0,N,N,1,0,0,0.0,0,0.0,0.0,Тульская,-1.00
1,50,5,Москва,35,7,Opel,77777,140.0,0,453000.0,N,M,0,0,0,0,1,1,0,0.0,0,4.0,0.0,Москва,-0.05
2,52,6,Москва,41,6,Kia,77777,129.0,0,647718.0,N,F,0,0,1L,1L,0,0,0,0.0,0,1.0,12518.0,Московская,-0.07
3,50,5,Москва,36,12,77777,77777,170.0,0,757795.0,N,M,0,0,0,0,1,1,0,0.0,0,6.0,15000.0,Москва,0.05
4,52,6,Санкт-Петербург,42,5,Renault,Logan,102.0,0,604500.0,171,F,0,0,N,N,0,0,0,0.0,0,0.0,50000.0,Ленинградская,0.17


In [30]:
X_train.corr()# матрица корреляций. нету сильных зависимостей между признаками

,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_SUM_INSURED,CLAIM_AVG_ACC_ST_PRD,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,POLICY_PRICE_CHANGE
POLICY_MIN_AGE,1.000000,0.025250,-0.063090,-0.047904,-0.026565,0.025296,-0.155315,-0.019457
POLICY_MIN_DRIVING_EXPERIENCE,0.025250,1.000000,-0.000683,0.048011,-0.007566,-0.091938,0.042772,-0.008690
VEHICLE_ENGINE_POWER,-0.063090,-0.000683,1.000000,0.779410,0.001887,-0.069905,-0.036795,0.041537
VEHICLE_SUM_INSURED,-0.047904,0.048011,0.779410,1.000000,0.007406,-0.310855,-0.062941,0.017505
CLAIM_AVG_ACC_ST_PRD,-0.026565,-0.007566,0.001887,0.007406,1.000000,-0.021835,-0.051096,0.014310
POLICY_YEARS_RENEWED_N,0.025296,-0.091938,-0.069905,-0.310855,-0.021835,1.000000,-0.060371,0.018342
POLICY_DEDUCT_VALUE,-0.155315,0.042772,-0.036795,-0.062941,-0.051096,-0.060371,1.000000,-0.040389
POLICY_PRICE_CHANGE,-0.019457,-0.008690,0.041537,0.017505,0.014310,0.018342,-0.040389,1.000000


# Векторизация

In [31]:
X_train.describe(include=[object])# смотрю статистику по кат-ым признакам

,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_IN_CREDIT,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,POLICY_HAS_COMPLAINTS,CLIENT_REGISTRATION_REGION
count,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407
unique,9,6,2,24,42,2,27,2,6,6,6,6,2,2,2,2,9
top,52,6,Москва,Kia,77777,0,77777,M,0,0,0,0,0,1,0,0,Санкт-Петербург
freq,19854,49797,40675,8803,28428,52605,30247,48923,54481,57885,35747,38588,56045,42577,77331,76842,31124


In [32]:
#созаю отдельный датафрейм со статистиками по кат-ым признакам
X_train_des = X_train.describe(include = [object])
X_train_des

,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_IN_CREDIT,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,POLICY_HAS_COMPLAINTS,CLIENT_REGISTRATION_REGION
count,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407,77407
unique,9,6,2,24,42,2,27,2,6,6,6,6,2,2,2,2,9
top,52,6,Москва,Kia,77777,0,77777,M,0,0,0,0,0,1,0,0,Санкт-Петербург
freq,19854,49797,40675,8803,28428,52605,30247,48923,54481,57885,35747,38588,56045,42577,77331,76842,31124


In [33]:
# выделюя бинарные и многоклассовые признаки используя датафрейм статистик
bin_col = [col for col in categorical_columns if X_train_des[col]['unique'] == 2] # бинарные признаки
non_bin_col = [col for col in categorical_columns if X_train_des[col]['unique'] > 2] # многоклассовые признаки
bin_col

['POLICY_BRANCH',
 'VEHICLE_IN_CREDIT',
 'INSURER_GENDER',
 'CLIENT_HAS_DAGO',
 'CLIENT_HAS_OSAGO',
 'POLICY_COURT_SIGN',
 'POLICY_HAS_COMPLAINTS']

In [34]:
non_bin_col

['POLICY_SALES_CHANNEL',
 'POLICY_SALES_CHANNEL_GROUP',
 'VEHICLE_MAKE',
 'VEHICLE_MODEL',
 'POLICY_INTERMEDIARY',
 'POLICY_CLM_N',
 'POLICY_CLM_GLT_N',
 'POLICY_PRV_CLM_N',
 'POLICY_PRV_CLM_GLT_N',
 'CLIENT_REGISTRATION_REGION']

In [35]:
X_train.head()

,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,CLIENT_REGISTRATION_REGION,POLICY_PRICE_CHANGE
0,77777,1,Москва,51,12,Land Rover,77777,245.0,0,1283202.0,77777,F,0,0,N,N,1,0,0,0.0,0,0.0,0.0,Тульская,-1.00
1,50,5,Москва,35,7,Opel,77777,140.0,0,453000.0,N,M,0,0,0,0,1,1,0,0.0,0,4.0,0.0,Москва,-0.05
2,52,6,Москва,41,6,Kia,77777,129.0,0,647718.0,N,F,0,0,1L,1L,0,0,0,0.0,0,1.0,12518.0,Московская,-0.07
3,50,5,Москва,36,12,77777,77777,170.0,0,757795.0,N,M,0,0,0,0,1,1,0,0.0,0,6.0,15000.0,Москва,0.05
4,52,6,Санкт-Петербург,42,5,Renault,Logan,102.0,0,604500.0,171,F,0,0,N,N,0,0,0,0.0,0,0.0,50000.0,Ленинградская,0.17


In [36]:
#по бинарным категориальным признакам прохожусь и меняю часто встречаюшийся элемент на 1 а все остальные на 0.
for col in bin_col:
    top = X_train_des[col]['top']
    mask_top_train = X_train[col] == top
    mask_top_test = X_test[col] == top
    X_train.loc[mask_top_train, col] = 0
    X_train.loc[np.logical_not(mask_top_train), col] = 1
    X_test.loc[mask_top_test, col] = 0
    X_test.loc[np.logical_not(mask_top_test), col] = 1

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [37]:
X_train[bin_col].describe()

,POLICY_BRANCH,VEHICLE_IN_CREDIT,INSURER_GENDER,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,POLICY_HAS_COMPLAINTS
count,77407.000000,77407.000000,77407.000000,77407.000000,77407.000000,77407.000000,77407.000000
mean,0.474531,0.320410,0.367977,0.275970,0.449959,0.000982,0.007299
std,0.499354,0.466637,0.482258,0.447005,0.497493,0.031319,0.085123
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
X_train.shape

(77407, 25)

In [39]:
X_train[bin_col].head()

,POLICY_BRANCH,VEHICLE_IN_CREDIT,INSURER_GENDER,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,POLICY_HAS_COMPLAINTS
0,0,0,1,1,1,0,0
1,0,0,0,1,0,0,0
2,0,0,1,0,1,0,0
3,0,0,0,1,0,0,0
4,1,0,1,0,1,0,0


In [40]:
# для многоклассовых признаков применяю дамми-кодирование как самое элементарное
X_train_non_bin = pd.get_dummies(X_train[non_bin_col])
len(X_train_non_bin.columns)

141

In [41]:
X_train_non_bin.head()

,POLICY_SALES_CHANNEL_6,POLICY_SALES_CHANNEL_10,POLICY_SALES_CHANNEL_50,POLICY_SALES_CHANNEL_52,POLICY_SALES_CHANNEL_53,POLICY_SALES_CHANNEL_54,POLICY_SALES_CHANNEL_55,POLICY_SALES_CHANNEL_59,POLICY_SALES_CHANNEL_77777,POLICY_SALES_CHANNEL_GROUP_1,POLICY_SALES_CHANNEL_GROUP_3,POLICY_SALES_CHANNEL_GROUP_4,POLICY_SALES_CHANNEL_GROUP_5,POLICY_SALES_CHANNEL_GROUP_6,POLICY_SALES_CHANNEL_GROUP_77777,VEHICLE_MAKE_77777,VEHICLE_MAKE_Audi,...,POLICY_PRV_CLM_N_77777,POLICY_PRV_CLM_N_N,POLICY_PRV_CLM_GLT_N_0,POLICY_PRV_CLM_GLT_N_1L,POLICY_PRV_CLM_GLT_N_1S,POLICY_PRV_CLM_GLT_N_2,POLICY_PRV_CLM_GLT_N_77777,POLICY_PRV_CLM_GLT_N_N,CLIENT_REGISTRATION_REGION_77777,CLIENT_REGISTRATION_REGION_N,CLIENT_REGISTRATION_REGION_Калужская,CLIENT_REGISTRATION_REGION_Ленинградская,CLIENT_REGISTRATION_REGION_Москва,CLIENT_REGISTRATION_REGION_Московская,CLIENT_REGISTRATION_REGION_Санкт-Петербург,CLIENT_REGISTRATION_REGION_Тверская,CLIENT_REGISTRATION_REGION_Тульская
0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [42]:
# дамми кодирование для тествых данных
X_test_non_bin = pd.get_dummies(X_test[non_bin_col])
len(X_test_non_bin.columns)

141

# Нормализация количественных признаков
### Числовые данные могут иметь большой разброс. Т.к. метод ближайших соседей чувствителен к масштабам, то провожу масштабирование. По каждому численному признаку вычесляю среднее, отнимаю от признака и делю на среднеквадратичное отклонение

In [43]:
X_train.head()

,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,CLIENT_REGISTRATION_REGION,POLICY_PRICE_CHANGE
0,77777,1,0,51,12,Land Rover,77777,245.0,0,1283202.0,77777,1,0,0,N,N,1,1,0,0.0,0,0.0,0.0,Тульская,-1.00
1,50,5,0,35,7,Opel,77777,140.0,0,453000.0,N,0,0,0,0,0,1,0,0,0.0,0,4.0,0.0,Москва,-0.05
2,52,6,0,41,6,Kia,77777,129.0,0,647718.0,N,1,0,0,1L,1L,0,1,0,0.0,0,1.0,12518.0,Московская,-0.07
3,50,5,0,36,12,77777,77777,170.0,0,757795.0,N,0,0,0,0,0,1,0,0,0.0,0,6.0,15000.0,Москва,0.05
4,52,6,1,42,5,Renault,Logan,102.0,0,604500.0,171,1,0,0,N,N,0,1,0,0.0,0,0.0,50000.0,Ленинградская,0.17


In [44]:
X_train_num = X_train[numerical_columns] # датафрейм содержит численные признаки обучающей выборки
X_train_num.shape

(77407, 8)

In [45]:
X_test_num = X_test[numerical_columns] # датафрейм содержит численные признаки тестовой выборки
X_test_num.shape

(19198, 8)

In [46]:
X_train_num.head()

,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_SUM_INSURED,CLAIM_AVG_ACC_ST_PRD,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,POLICY_PRICE_CHANGE
0,51,12,245.0,1283202.0,0.0,0.0,0.0,-1.00
1,35,7,140.0,453000.0,0.0,4.0,0.0,-0.05
2,41,6,129.0,647718.0,0.0,1.0,12518.0,-0.07
3,36,12,170.0,757795.0,0.0,6.0,15000.0,0.05
4,42,5,102.0,604500.0,0.0,0.0,50000.0,0.17


In [47]:
# от каждого признака отнимаю среднее и делю на ср.кв.отклонение
X_train_num = (X_train_num - X_train_num.mean()) / X_train_num.std()
X_train_num.describe()

,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_SUM_INSURED,CLAIM_AVG_ACC_ST_PRD,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,POLICY_PRICE_CHANGE
count,7.740700e+04,7.740700e+04,7.740700e+04,7.740700e+04,7.740700e+04,7.740700e+04,7.740700e+04,7.740700e+04
mean,-4.144974e-16,-5.251676e-17,-4.353875e-16,1.050504e-16,-6.524495e-16,1.368981e-15,-3.579873e-16,-1.899140e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.298343e+00,-1.748855e-01,-2.843376e+00,-1.425290e+00,-2.176408e-01,-9.333436e-01,-5.736841e-01,-1.154193e+00
25%,-8.051465e-01,-1.350505e-01,-5.764088e-01,-6.134378e-01,-2.176408e-01,-9.333436e-01,-5.736841e-01,-1.630055e-01
50%,-1.518730e-01,-1.051743e-01,-1.525042e-01,-2.509500e-01,-2.176408e-01,-3.091820e-01,-5.736841e-01,4.000876e-02
75%,6.880501e-01,-8.027747e-02,3.082616e-01,2.689445e-01,-2.176408e-01,3.149796e-01,3.880629e-01,9.971883e-02
max,4.047743e+00,9.858543e+00,3.401789e+01,1.231631e+01,4.087873e+01,5.308272e+00,1.105124e+01,7.171598e+01


In [48]:
# от каждого признака отнимаю среднее и делю на ср.кв.отклонение
X_test_num = (X_test_num - X_test_num.mean()) / X_test_num.std()
X_test_num.describe()

,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_SUM_INSURED,CLAIM_AVG_ACC_ST_PRD,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,POLICY_PRICE_CHANGE
count,1.919800e+04,1.919800e+04,1.919800e+04,1.919800e+04,1.919800e+04,1.919800e+04,1.919800e+04,1.919800e+04
mean,-1.734904e-16,1.709314e-17,1.407158e-15,-1.151514e-16,1.480535e-15,3.874157e-16,-3.465760e-16,-1.238931e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.299175e+00,-1.735276e-01,-2.873566e+00,-1.404518e+00,-2.303535e-01,-9.317445e-01,-5.723044e-01,-1.034548e+00
25%,-7.910807e-01,-1.327658e-01,-5.766364e-01,-6.051208e-01,-2.303535e-01,-9.317445e-01,-5.723044e-01,-1.589910e-01
50%,-1.312895e-01,-1.072897e-01,-1.471293e-01,-2.447727e-01,-2.303535e-01,-3.074815e-01,-5.723044e-01,3.320444e-02
75%,7.170134e-01,-7.671840e-02,3.197263e-01,2.477678e-01,-2.303535e-01,3.167814e-01,3.798094e-01,8.659206e-02
max,3.827457e+00,1.007296e+01,8.050855e+00,1.002098e+01,2.260648e+01,5.310885e+00,8.751365e+00,4.933133e+01


In [49]:
# объединяю все данные в единый числовой датафрейм
X_train = pd.concat((X_train_num, X_train[bin_col], X_train_non_bin), axis = 1)

In [50]:
X_train.shape

(77407, 156)

In [51]:
X_train.head()

,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_SUM_INSURED,CLAIM_AVG_ACC_ST_PRD,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,POLICY_PRICE_CHANGE,POLICY_BRANCH,VEHICLE_IN_CREDIT,INSURER_GENDER,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,POLICY_HAS_COMPLAINTS,POLICY_SALES_CHANNEL_6,POLICY_SALES_CHANNEL_10,...,POLICY_PRV_CLM_N_77777,POLICY_PRV_CLM_N_N,POLICY_PRV_CLM_GLT_N_0,POLICY_PRV_CLM_GLT_N_1L,POLICY_PRV_CLM_GLT_N_1S,POLICY_PRV_CLM_GLT_N_2,POLICY_PRV_CLM_GLT_N_77777,POLICY_PRV_CLM_GLT_N_N,CLIENT_REGISTRATION_REGION_77777,CLIENT_REGISTRATION_REGION_N,CLIENT_REGISTRATION_REGION_Калужская,CLIENT_REGISTRATION_REGION_Ленинградская,CLIENT_REGISTRATION_REGION_Москва,CLIENT_REGISTRATION_REGION_Московская,CLIENT_REGISTRATION_REGION_Санкт-Петербург,CLIENT_REGISTRATION_REGION_Тверская,CLIENT_REGISTRATION_REGION_Тульская
0,0.781375,-0.115133,1.672128,0.440859,-0.217641,-0.933344,-0.573684,-1.154193,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,-0.711822,-0.140030,-0.263088,-0.766496,-0.217641,1.563303,-0.573684,-0.019701,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,-0.151873,-0.145009,-0.465825,-0.483320,-0.217641,-0.309182,0.630231,-0.043585,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,-0.618497,-0.115133,0.289831,-0.323236,-0.217641,2.811626,0.868936,0.099719,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,-0.058548,-0.149989,-0.963452,-0.546171,-0.217641,-0.933344,4.235051,0.243023,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [52]:
# объединяю все данные в единый числовой датафрейм
X_test = pd.concat((X_test_num, X_test[bin_col], X_test_non_bin), axis = 1)

In [53]:
#все элементы в обоих датафреймах привожу к типу float
X_train = pd.DataFrame(X_train, dtype = float)
X_test = pd.DataFrame(X_test, dtype = float)

# Строим различные модели для выбора наиболее подходящего

# Метод ближайших соседей

In [54]:
num_neig = [3,5]# список для параметра - кол-о соседей. т.к. бинарная классификация выбираем нечетное кол-о  соседей
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid= {'n_neighbors': num_neig})
grid.fit(X_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None, param_grid={'n_neighbors': [3, 5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [55]:
cv_err = 1 - grid.best_score_
best_num_neig = grid.best_estimator_.n_neighbors
print('Ошибка перекрестной проверки:', cv_err*100, '%\n', 'Оптимальное кол-о соседей:', best_num_neig)

Ошибка перекрестной проверки: 37.29766041830842 %
 Оптимальное кол-о соседей: 5


In [56]:
#обучаю модель на оптимальном кол-е соседей
knn = KNeighborsClassifier(n_neighbors=best_num_neig)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [57]:
'''
выбрал 2 метрики оценки качества предсказания:
1)среднее по количеству не совпадений
2)АUC
'''
error_train = np.mean(y_train != knn.predict(X_train))
error_train_AUC = roc_auc_score(y_train, knn.predict_proba(X_train)[:, 1])
print('Ошибка на обучающей выборке:', error_train*100, '%')
print("AUC на обучающей выборке : {:.3f}".format(error_train_AUC))

Ошибка на обучающей выборке: 25.467980931957058 %
AUC на обучающей выборке : 0.809


# Метод случайного леса

In [58]:
'''
Для выбора кол-а деревьев в модели будем использовать кросс-валидацию 2х моделей
использую 2 модели со 100 деревьями и с 1000ю деревьями и глубиной 15
Затем та модель у которой среднее по оценке будет лучше и примем в качестве основной
'''
rand_forest_100 = RandomForestClassifier(n_estimators= 100, random_state= 42)
rand_forest_1000 = RandomForestClassifier(n_estimators= 1000, max_depth= 15, random_state= 42)
scores100 = cross_val_score(rand_forest_100, X_train, y_train, cv = 5)
scores1000 = cross_val_score(rand_forest_1000, X_train, y_train, cv = 5)

In [59]:
print(scores100)
print(scores1000)
print(pd.Series(scores100).mean())
print(pd.Series(scores1000).mean())

[0.70212491 0.70441186 0.70738324 0.70408888 0.70060074]
[0.69973519 0.70395969 0.70531619 0.70583296 0.69659583]
0.7037219268804034
0.7022879727357809


In [60]:
rand_forest_100.fit(X_train, y_train)
error_train_100 = np.mean(y_train != rand_forest_100.predict(X_train))
print('100 деревьев. Ошибка на обучающей выборке:', error_train)
print("100 деревьев. AUC на обучающей выборке : {:.3f}".format(
    roc_auc_score(y_train, rand_forest_100.predict_proba(X_train)[:, 1])))

100 деревьев. Ошибка на обучающей выборке: 0.25467980931957057
100 деревьев. AUC на обучающей выборке : 1.000


### В итоге из 2х моделей("соседи" и "случайный лес") выигрывает "случайный лес" из 100 деревьев.

In [61]:
y_predict = rand_forest_100.predict(X_test)

In [62]:
y_predict[:5]

array([1, 1, 0, 0, 1], dtype=int64)

In [63]:
#преобразуем наши данные в датафрейм для записи в файл
y_predict_s = pd.DataFrame({'POLICY_ID': pd.Series(X_test.index), 'POLICY_IS_RENEWED':pd.Series(y_predict)})

In [64]:
y_predict_s.head()

,POLICY_ID,POLICY_IS_RENEWED
0,9,1
1,11,1
2,12,0
3,21,0
4,36,1


In [65]:
y_predict_s.shape

(19198, 2)

In [66]:
y_predict_s.to_csv('prediction_data.csv')